<a href="https://colab.research.google.com/github/Azazftb/lung-detect-CT/blob/main/lung_detect_CT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>